In [1]:
import json
import numpy as np
from tqdm import tqdm
import re
import os
import json
import random

In [2]:
total_expected_qas = 5

In [3]:
def sensor_subsampled_string(data, n=60):
    if len(data)/n>10:
        print(f"High compression: {len(data)/n}")
    indices = np.round(np.linspace(0, len(data) - 1, n)).astype(int)
    return str([list(np.round(data[idx],6)) for idx in indices])

In [4]:
# q_regex = re.compile(r"^\W{0,5}\d{1,2}\.[\n\r\s]+|^\W{0,5}Question\s?\d{0,2}\W{0,5}|^\W{0,5}Q\s?\d{1,2}\W{0,5}|^\W{0,5}\d{1,2}\.[\n\r\s]+Question\W{0,5}")
# a_regex = re.compile(r"^\W{0,5}\d{0,2}\W{0,5}Answer\s?\d{0,2}\W{0,5}|^-|^\d{0,2}\W{0,5}A\s?\d{0,2}\W{0,5}:")

q_regex = re.compile(r"^\W{0,5}\d{0,2}\W{0,5}Q\d{0,2}\W{0,5}:\W{0,5}")
a_regex = re.compile(r"^\W{0,5}A\d{0,2}\W{0,5}:\W{0,5}")

def tuplize_qa_pair(q, a):
    a=a.strip()
    if len(a)>10:
        if a_regex.match(a):
            a_preface, a_content = a_regex.split(a,1)
        elif a[0].isalpha():
            a_content = a
        else:
            raise RuntimeError(f"Answer regex incomplete: {q}\nANSWER:{a}")
        
        a_content = a_content.strip()
        if len(a_content)<10:
            print(f"Small answer dropping: {a_content}")
            return None
        return (q, a_content)
    else:
        if len(a)<10 and len(a)>0:
            print(f"Small answer dropping: {a}")
        return None


def one_example(sample):
    sample_dict = json.loads(sample)
    qa_lines = sample_dict["messages"][2]["content"].split("\n")
    qa_pair_list = []
    q_prefix = ''
    q_content = ''
    a = ''
    missing = 0
    for line in qa_lines:
        line = line.strip()
        if q_regex.match(line) and not a_regex.match(line):
            old_q_prefix = q_prefix
            old_q_content = q_content
            assert len(q_regex.findall(line))==1, q_regex.findall(line)
            q_prefix = q_regex.findall(line)[0]
            q_content  = q_regex.split(line,1)[1]
            if q_prefix == old_q_prefix and re.match(r"\d",q_prefix):
                # Answer recovery when q and a are both 1. 2. etc
                a+=q_content+"\n"
                q_content = old_q_content
            else:
                qa_pair = tuplize_qa_pair(q_content, a)
                if qa_pair is not None:
                    qa_pair_list.append(qa_pair)
                a = ''
        else:
            a+=line+'\n'
    qa_pair = tuplize_qa_pair(q_content, a)
    if qa_pair is not None:
        qa_pair_list.append(qa_pair)
    return qa_pair_list, total_expected_qas-len(qa_pair_list)

In [5]:
image_dir = "/hdd/LLM/limuBERT_data/train_llasa/images"

In [6]:
def limubert_normalize(data):
    data[:, :, :3] = data[:, :, :3]/9.8
    return data

In [7]:
training_jsonl_filepath = "llasa_training_v2_raw.jsonl"

root_data_dir = "/hdd/LLM/limuBERT_data/extracted_data"

data_file_name = "data_20_120.npy"
label_file_name = "label_20_120.npy"


with open('/hdd/LLM/limuBERT_data/dataset_activity_label.json') as json_file:
    dataset_activity_label_dict = json.load(json_file)


last_axis_dict = {
    "hhar": 2, "motion":0, "shoaib":0, "uci":0
}
datasets = last_axis_dict.keys()

all_qa_pair = []
qa_pair_list = []
for dataset in datasets:
    p = 0
    sample_count = 0
    with open(f"/hdd/LLM/opensqa_10hz_{dataset}_v2.jsonl","r") as f:
        samples = f.readlines()
    sensor_data = np.load(os.path.join(root_data_dir, dataset, data_file_name))
    sensor_data = limubert_normalize(sensor_data)
    
    for sample in tqdm(samples):
        qa_pairs, missing = one_example(sample)
        if missing<0:
            print(f"WARNING! Extra: {qa_pairs} Missing: {missing}")
        if missing>5:
            print(f"{missing} missing:{json.loads(sample)['messages'][2]['content']}")
            print(qa_pairs)
            if missing<total_expected_qas:
                print("Potential corner case")
        else:
            
            accl_gyro = sensor_data[sample_count][:,:6] # accl, gyro together
            accl, gyro = sensor_data[sample_count][:, 0:3], sensor_data[sample_count][:, 3:6]
            accl, gyro = accl.tolist(), gyro.tolist()
            accl_str = sensor_subsampled_string(accl)
            gyro_str = sensor_subsampled_string(gyro)
            
            all_qa_pair.extend(qa_pairs)
            for i, (q, a) in enumerate(qa_pairs):
                sensor_sample_id = f"{sample_count}_{i}"
                sensor_sample_filepath = os.path.join(image_dir , f'{dataset}/sensor_image_{sample_count}.npy')
                np.save(sensor_sample_filepath, accl_gyro)
                qa_pair = {
                    "id": sensor_sample_id,
                    "image": sensor_sample_filepath, # Sensor csv
                    "conversations": [
                        {
                            "from": "human",
                            "value": (
                                f"<image>\n{q} "
                                f"Gyroscope: {gyro_str}"
                                f"Accelerometer: {accl_str}"
                            )
                        }, # question
                        {"from": "gpt", "value": a}, # answers
                    ],
                }
                qa_pair_list.append(qa_pair)
                
        sample_count += 1
        p+=missing
    print(f"Missing: {p}, {100*p/(len(sensor_data)*total_expected_qas)}%")

 38%|███▊      | 3486/9166 [00:06<00:09, 582.53it/s]

WARNING! Extra: [("Based on the gyroscope data, how do fluctuations in the z-axis orientation reflect the subject's adaptive strategies during the descent of stairs, and what does the variance in these readings indicate about the stability of the individual's motion?", 'Here are five complex questions and their corresponding answers, derived logically and analytically from the provided IMU dataset and narration:'), ("Analyze the relationship between the accelerometer readings’ peaks and valleys in the z-axis with respect to the gyroscope's z-axis data, particularly in the contexts of body weight distribution during descending steps. What does this indicate about the biomechanics involved?", "The gyroscope data shows a downward trend in the z-axis, consistently reflecting the subject's descent from the stairs. The initial fluctuations near the beginning indicate adaptive strategies as the subject adjusts their body posture and orientation to find balance—the presence of high variance in

100%|██████████| 9166/9166 [00:16<00:00, 561.38it/s]


Missing: 4, 0.008727907484180668%


 46%|████▌     | 2095/4534 [00:03<00:04, 532.93it/s]

...

Missing: 16, 0.030476190476190476%


 48%|████▊     | 1003/2088 [00:01<00:01, 547.38it/s]

WARNING! Extra: [('Analyzing the gyroscopic data, how can you differentiate between the rotational movements of limbs involved in stair climbing during the initial phase versus the continuous climbing phase based on the provided data?', 'Here are five complex question-answer pairs regarding the IMU dataset related to stair climbing:'), ('How do the accelerometer readings reflect the impact forces during foot strikes on the steps, and what specific values indicate noteworthy transitions in weight transfer throughout the climb?', 'During the initial phase of stair climbing, the gyroscopic data shows moderate rotation on the z-axis (e.g., 0.272293) when the foot first touches the stair, indicating the beginning of a twisting motion required to lift the leg. As the climbing continues, these values become more dynamic, reflecting greater oscillations in all axes, specifically showing declines in the y-axis indicative of counterclockwise rotation as the body dynamically moves in rhythm with 

100%|██████████| 2088/2088 [00:03<00:00, 559.66it/s]

Missing: -1, -0.009578544061302681%


In [8]:
all_qa_pair[50000]

('How do the minimum and maximum values of the gyroscope and accelerometer complement the understanding of the subject’s activity? Provide a detailed comparison.',
 'The minimum gyroscope values are predominantly low, with maximums reaching approximately 0.03 rad/s, indicating negligible rotational movement typical of sitting. In contrast, accelerometer readings cluster around 3.4 to 3.5 m/s², akin to gravitational force. This range of accelerometer readings indicates the body’s weight is effectively vertical, further substantiating that the subject is seated. Together, the data suggest that the individual is not engaged in dynamic activities but rather remains in a stable seated position with slight natural micro-movements, which align with the activity context.')

In [9]:
len(qa_pair_list)

131407

In [10]:
sensor_locations = ['ankle', 'chest', 'hand']

with open(f"/hdd/LLM/opensqa_10hz_pamap2_v2.jsonl","r") as f:
    opensqa_samples = f.readlines()

opensqa_sample_idx = 0
for subject_id in range(1,9):
    label = np.load(f'/hdd/LLM/PAMAP2/original/subject10{subject_id}.dat/label.npy')
    for location in sensor_locations:
        data = np.load(f'/hdd/LLM/PAMAP2/original/subject10{subject_id}.dat/{location}.npy')
        data = limubert_normalize(data)
        
        for sample_index in tqdm(range(len(data))):
            sample_data = data[sample_index]
            if label[sample_index]=='0':
                continue
            
            opensqa_sample = opensqa_samples[opensqa_sample_idx]
            
            qa_pairs, missing = one_example(opensqa_sample)
            if missing<0:
                print(f"WARNING! Extra: {qa_pairs} Missing: {missing}")
                opensqa_sample_idx += 1
                continue
            if missing>5:
                print(f"{missing} missing:{json.loads(sample)['messages'][2]['content']}")
                print(qa_pairs)
                if missing<total_expected_qas:
                    print("Potential corner case")
                opensqa_sample_idx += 1
                continue
            
            all_qa_pair.extend(qa_pairs)
            
            accl_gyro = sample_data[:,:6]
            accl, gyro = sample_data[:, 0:3], sample_data[:, 3:6]
            accl, gyro = accl.tolist(), gyro.tolist()
            accl_str = sensor_subsampled_string(accl)
            gyro_str = sensor_subsampled_string(gyro)
            sensor_sample_filepath = os.path.join(
                image_dir , f'pamap2/sensor_image_{subject_id}_{location}_{opensqa_sample_idx}.npy'
            )
            for i, (q, a) in enumerate(qa_pairs):
                sensor_sample_id = f"pamap2_{opensqa_sample_idx}_{i}"
                np.save(sensor_sample_filepath, accl_gyro)
                qa_pair = {
                    "id": sensor_sample_id,
                    "image": sensor_sample_filepath, # Sensor csv
                    "conversations": [
                        {
                            "from": "human",
                            "value": (
                                f"<image>\n{q} "
                                f"Gyroscope: {gyro_str}"
                                f"Accelerometer: {accl_str}"
                            )
                        }, # question
                        {"from": "gpt", "value": a}, # answers
                    ],
                }
                qa_pair_list.append(qa_pair)
            opensqa_sample_idx += 1
            
            

  1%|▏         | 8/627 [00:00<00:14, 42.87it/s]

High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8


  2%|▏         | 13/627 [00:00<00:14, 41.41it/s]

High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8


  3%|▎         | 18/627 [00:00<00:19, 30.71it/s]

...

High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8


 93%|█████████▎| 633/680 [00:17<00:00, 73.96it/s]

High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8


100%|██████████| 680/680 [00:18<00:00, 37.10it/s]

High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8
High compression: 34.8


In [11]:
with open(training_jsonl_filepath, "w") as outfile:
    json.dump(qa_pair_list, outfile)

In [12]:
len(qa_pair_list)

179727

In [13]:
qa_pair_list[0]

{'id': '0_0',
 'image': '/hdd/LLM/limuBERT_data/train_llasa/images/hhar/sensor_image_0.npy',
 'conversations': [{'from': 'human',
   'value': '<image>\nHow does the accelerometer data confirm the phenomenon of postural sway in a standing human, and what do the fluctuations around a gravitational force of approximately 8.0 m/s² indicate? Gyroscope: [[-0.00943, -0.008636, -0.001587], [-0.014023, -0.007423, -0.001432], [-0.015732, -0.017822, 0.00386], [-0.003876, -0.051468, -0.016327], [-0.002511, -0.063288, 0.004157], [-0.016062, -0.087823, 0.027019], [-0.010498, -0.12442, 0.026291], [-0.017975, -0.110001, 0.016357], [-0.020334, -0.09462, 0.00119], [-0.024918, -0.051788, 0.010056], [-0.017975, -0.055634, 0.022446], [-0.022675, 0.004288, 0.001083], [-0.02182, -0.001801, 0.027252], [-0.006643, -0.002179, 0.019154], [-0.007614, -0.0065, 0.002151], [0.01503, -0.019211, -0.041428], [0.025711, -0.017181, -0.037903], [0.00531, -0.004578, 0.000656], [0.000737, -0.009559, 0.004492], [-0.012955, 0

In [14]:
qa_pair_list[177777]

{'id': 'pamap2_9281_0',
 'image': '/hdd/LLM/limuBERT_data/train_llasa/images/pamap2/sensor_image_8_hand_9281.npy',
 'conversations': [{'from': 'human',
   'value': '<image>\nAnalyze the frequency characteristics of the gyroscope data using its FFT. What does the frequency spectrum suggest about the overall activity of the individual during this seated event? Gyroscope: [[array([ 1.019673, -0.122305,  0.096825,  0.013337, -0.005905,  0.002647]), array([ 1.021526, -0.12022 ,  0.097288,  0.      , -0.005651,  0.004123]), array([ 1.01921 , -0.120915,  0.097751, -0.006618, -0.00224 ,  0.004581])], [array([ 1.260114, -0.562418,  0.152881,  0.10853 , -0.577573,  0.153938]), array([ 0.783865, -0.448916, -0.058373, -0.116217, -1.086882,  0.182496]), array([ 1.021989, -0.528599, -0.210791, -0.206676, -1.296154,  0.053349])], [array([ 0.688893, -0.228859,  0.065785, -0.221031,  0.831286,  0.076562]), array([ 0.654611, -0.298814,  0.167475, -0.14844 ,  0.63362 ,  0.022907]), array([ 0.769967, -0.3

In [15]:
qa_pair_list[176777]

{'id': 'pamap2_9081_0',
 'image': '/hdd/LLM/limuBERT_data/train_llasa/images/pamap2/sensor_image_8_chest_9081.npy',
 'conversations': [{'from': 'human',
   'value': '<image>\nAnalyze the gyroscope data to determine the degree of upper body rotation during this walking activity. How can you assess the stability of the walking pattern based on this rotational data? Gyroscope: [[array([ 1.019673, -0.122305,  0.096825,  0.013337, -0.005905,  0.002647]), array([ 1.021526, -0.12022 ,  0.097288,  0.      , -0.005651,  0.004123]), array([ 1.01921 , -0.120915,  0.097751, -0.006618, -0.00224 ,  0.004581])], [array([ 1.260114, -0.562418,  0.152881,  0.10853 , -0.577573,  0.153938]), array([ 0.783865, -0.448916, -0.058373, -0.116217, -1.086882,  0.182496]), array([ 1.021989, -0.528599, -0.210791, -0.206676, -1.296154,  0.053349])], [array([ 0.688893, -0.228859,  0.065785, -0.221031,  0.831286,  0.076562]), array([ 0.654611, -0.298814,  0.167475, -0.14844 ,  0.63362 ,  0.022907]), array([ 0.769967,